In [ ]:
import torch
import os
import torch.nn as nn
from torch.utils.data import DataLoader
from utils.utils import safe_load_mesh
import pickle
import sys
from models.vae_models import VAE, VAE2, KL_divergence
from models.discriminators import PointNet2, GHD_Reconstruct
from models.losses import wgan_gradient_penalty
from torch_geometric.data import Data
import torch.nn.functional as F
import wandb
import matplotlib.pyplot as plt
from models.utils import save_models, load_models, plot_wandb
from sklearn.decomposition import PCA
import numpy as np
import pytorch3d
from pytorch3d.loss import mesh_laplacian_smoothing, mesh_normal_consistency
from losses.mesh_loss import Rigid_Loss
from models.vae_datasets import GHDDataset as Dataset
from models.utils import get_fig, get_fig_advanced
from IPython.display import Image

In [ ]:
# data conf
root = '/media/yaplab/"HDD Storage"/wenhao/ghb_aneurysms'
ghd_chk_root = os.path.join(root, 'checkpoints/fit_mca_stable')
ghd_run = 'vanilla'
ghd_chk_name = 'ghb_fitting_checkpoint_5.pkl'
eigen_chk = os.path.join(root, "checkpoints/canonical_typeB_144.pkl")
alignment_root = os.path.join(root, 'checkpoints/align_mca_female')
canonical_name = 'canonical_typeB'
canonical_Meshes = safe_load_mesh(os.path.join(alignment_root, canonical_name, 'part_aligned_updated.obj'))
cases = [case for case in os.listdir(ghd_chk_root) if os.path.isdir(os.path.join(ghd_chk_root, case)) and case != canonical_name]
# model conf
epochs = 1001
hidden_dim = 108
latent_dim = 108
batch_size = 128
mode = 'train'
device = torch.device('cuda:0')
use_norm = True
use_reg = True
rigidloss = Rigid_Loss(canonical_Meshes.to(device))
use_discriminator = True
pure_gan = False
reload_epoch = 10000
# meta
meta = 'stable_108_h108_fixreg'
withscale = False
log_path = os.path.join("./checkpoints/paper/VAE", meta)
get_gif = False
save_objs = False

In [ ]:
from models.mesh_plugins import MeshPlugins, MeshRegulizer
import pyvista as pv
pv.set_jupyter_backend('html')
pv.start_xvfb()

cep_chk = "./data/excision_registration/combined_stable/canonical_typeB/diff_centreline_checkpoint.pkl"
trimmed_mesh_path = os.path.join(os.getcwd(), "data/excision_registration/combined_stable/canonical_typeB/part_trimmed_short.obj")
wave_based_trimming = False
mesh_plugin = MeshPlugins(canonical_Meshes, cep_chk, 
                          max_loops=[15, 13, 12], loop_start=[11, 9, 8], 
                          trimmed_mesh_path=trimmed_mesh_path, wave_based_trimming=wave_based_trimming)
print(mesh_plugin.cap_faces)
print("visualize mesh regulizer for calibration")
mesh_regulizer = MeshRegulizer(mesh_plugin, device=device, visualize=False)
# trumpet_loss = mesh_regulizer.trumpet_loss(canonical_Meshes)

In [ ]:
# dataset
ghd_reconstruct = GHD_Reconstruct(canonical_Meshes, eigen_chk, num_Basis=12**2, device=device)
dataset = Dataset(ghd_chk_root, ghd_run, ghd_chk_name, ghd_reconstruct, cases, withscale=withscale, normalize=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
mean, std = dataset.get_mean_std()
# model
generator = VAE(dataset.get_dim(), hidden_dim, latent_dim, withscale=withscale).to(device)
discriminator = PointNet2(use_norm=use_norm).to(device)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.001, betas=(0.9, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=2000, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=2000, gamma=0.5)
# reload
generator, discriminator, optimizer_G, optimizer_D, epoch_ = load_models(generator, discriminator, optimizer_G, optimizer_D, log_path, reload_epoch, generator_only=True)
generator.eval()

In [ ]:
"""mesh render"""
from render.mesh_renders import MeshRender

out_dir = "./render/gifs"
mesh_render = MeshRender(out_dir=out_dir, device=device)
mesh_render.init_mesh_render(image_size=1024, dist=1, elev=45, azim=90)

In [ ]:
# %matplotlib widget
# unconditional generation
label='unconditional_generation'
B = 32
z = torch.randn(B, latent_dim).to(device) * 1.0
if withscale:
    ghd_fake, scale_fake = generator.decode(z, False)
    scale_fake = dataset.denorm_scale(scale_fake)
else:
    ghd_fake = generator.decode(z, True)
    scale_fake = None
data_fake = ghd_reconstruct.forward(ghd_fake, mean, std, use_norm)
# fig = get_fig(ghd_reconstruct, data_fake, 4, Title="Unconditional Generation")
# plt.show(fig)
fig2 = get_fig_advanced(ghd_reconstruct, data_fake, 8, Title=None, mesh_plugin=mesh_plugin, plot_tangent=False, scale=scale_fake)
save_path = "render/images/unconditional/{}/{}.png".format(label, meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show(fig2)

# create gifs
meshes = ghd_reconstruct.ghd_forward_as_Meshes(ghd_fake * std.to(device) + mean.to(device), denormalize_shape=False, trimmed_faces=mesh_plugin.trimmed_faces)

if get_gif:
    # meshes = ghd_reconstruct.ghd_forward_as_Meshes(ghd_fake * std.to(device) + mean.to(device), denormalize_shape=False, trimmed_faces=None)
    out_dir = "render/wepage/unconditional"
    mesh_render.render_rotational_gif(meshes, dual_texture=False, n_row=8, fps=36, out_dir=out_dir, label=label)
try:
    # display(Image(filename=f"./render/gifs/{label}.gif"))
    pass
except:
    pass
if save_objs:
    save_dir = "render/meshes/unconditional/{}/{}".format(label, meta)
    os.makedirs(os.path.dirname(save_dir), exist_ok=True)
    mesh_render.save_objs(meshes, label, save_dir=save_dir, add_random_string=False)
# ur = input("Please input UR: ")
# dl = input("Please input DL, ranging from 1-3: ")
# ur = "none"
# dl = "none"

In [ ]:
# Evaluator
from evaluators import GenerationEvaluator, save_metrics_to_excel

evaluator = GenerationEvaluator(dataset, ghd_reconstruct, mesh_plugin, dmm_calculator=None, root=os.getcwd())
B = 512
z = torch.randn(B, latent_dim).to(device)
ghd_fake = generator.decode(z, True)
ghd_real = next(iter(dataloader)).to(device)
if withscale:
    ghd_real, scale_real = ghd_real[:, :-1], ghd_real[:, -1:]
else:
    scale_real = None
ghd_recon, _, _ = generator(ghd_real, scale_real)

input_real, input_fake = evaluator.preprocess_ghd(ghd_real, ghd_fake)
fpd = evaluator.compute_fpd(input_real, input_fake)
kpd = evaluator.compute_kpd(input_real, input_fake)
tmd = evaluator.compute_tmd(input_fake)
tmd_real = evaluator.compute_tmd(input_real)
cd_v, cd_n = evaluator.compute_cd(ghd_real, ghd_recon, mean, std)
print(f"FPD: {fpd}, KPD: {kpd}, TMD: {tmd}, CD_V: {cd_v}, CD_N: {cd_n}, TMD_real: {tmd_real}")

# save csv
file_path = "metrics.xlsx"
sheet_name = "unconditional_GHD_VAE"
run_type = "unconditional"
metrics = {"latent_dim": latent_dim, "hidden_dim": hidden_dim, "withscale": withscale, "FPD": fpd, "KPD (10^3)": 1e3*kpd, "TMD (10^4)": 1e4*tmd, "ur": ur, "dl": dl,
           "CD_V (10^4)": 1e4*cd_v, "CD_N (10^2)": 1e2*cd_n}
# save_metrics_to_excel(file_path, sheet_name, meta, run_type, metrics)

In [ ]:
# Evaluator PCA
from evaluators import GenerationEvaluator, save_metrics_to_excel

evaluator = GenerationEvaluator(dataset, ghd_reconstruct, mesh_plugin, dmm_calculator=None, root=os.getcwd())
B = 512
z = torch.randn(B, latent_dim).to(device)
ghd_fake = generator.decode(z, True)
ghd_real = next(iter(dataloader)).to(device)
if withscale:
    ghd_real, scale_real = ghd_real[:, :-1], ghd_real[:, -1:]
else:
    scale_real = None
ghd_recon, _, _ = generator(ghd_real, scale_real)

input_real, input_fake = evaluator.preprocess_ghd(ghd_real, ghd_fake)
input_fake = torch.load("./render/meshes/pca/pca_generation.pth")
input_fake = torch.Tensor(input_fake).to(device)
fpd = evaluator.compute_fpd(input_real, input_fake)
kpd = evaluator.compute_kpd(input_real, input_fake)
tmd = evaluator.compute_tmd(input_fake)
cd_v, cd_n = evaluator.compute_cd(ghd_real, ghd_recon, mean, std)
print(f"FPD: {fpd}, KPD: {kpd}, TMD: {tmd}, CD_V: {cd_v}, CD_N: {cd_n}")

# save csv
file_path = "metrics.xlsx"
sheet_name = "unconditional_GHD_VAE"
run_type = "unconditional"
metrics = {"latent_dim": latent_dim, "hidden_dim": hidden_dim, "withscale": withscale, "FPD": fpd, "KPD (10^3)": 1e3*kpd, "TMD (10^4)": 1e4*tmd, "ur": ur, "dl": dl,
           "CD_V (10^4)": 1e4*cd_v, "CD_N (10^2)": 1e2*cd_n}
# save_metrics_to_excel(file_path, sheet_name, "PCA", run_type, metrics)

In [ ]:
"""plot Umap"""
import umap
from sklearn.cluster import KMeans

style = 'fake'

# extract latent
B = 1024
z = torch.randn(B, latent_dim).to(device)
if withscale:
    ghd_fake, scale_fake = generator.decode(z, False)
else:
    ghd_fake = generator.decode(z, True)
    scale_fake = None
mu, logvar = generator.encode(ghd_fake, scale_fake)
z_fake = mu
mu, logvar = generator.encode(ghd_real, scale_real)
z_real = mu

if style == 'fake':
    fit = umap.UMAP(n_neighbors=20)
    u = fit.fit_transform(z_fake.detach().cpu().numpy())
else:
    fit = umap.UMAP(n_neighbors=2)
    u = fit.fit_transform(z_real.detach().cpu().numpy())

# cluster
kmeans = KMeans(n_clusters=8, random_state=55).fit(u)
labels_fake = kmeans.labels_
u_centers = kmeans.cluster_centers_

# plot UMAP with different colors for each cluster
# Define markers
markers = ['o', '^', '*', '+', 'h']
unique_labels = np.unique(labels_fake)
colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

# Plot UMAP with fixed markers and unique colors
for i, label in enumerate(unique_labels):
    mask = (labels_fake == label)  # Select points for the current cluster
    plt.scatter(
        u[mask, 0], u[mask, 1],
        alpha=0.3,
        marker=markers[i % len(markers)],  # Assign marker
        color=colors[i]  # Assign color
    )
    plt.scatter(u_centers[i, 0], u_centers[i, 1], c=colors[i], marker="2", s=100)
    # plt.text(u_centers[i, 0], u_centers[i, 1], str(i), fontsize=12, ha='right')
save_path = "render/images/unconditional/{}/{}.png".format("umap", meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show()

# Reverse transform u_centers to get the corresponding z values
z_cluster = torch.Tensor(fit.inverse_transform(u_centers)).to(device)
if withscale:
    ghd_cluster, scale_cluster = generator.decode(z_cluster, False)
    scale_cluster = dataset.denorm_scale(scale_cluster)
else:
    ghd_cluster = generator.decode(z_cluster)
    scale_cluster = None
data_cluster = ghd_reconstruct.forward(ghd_cluster, mean, std, use_norm)
fig_cluster = get_fig_advanced(ghd_reconstruct, data_cluster, 4, Title="Clusters", mesh_plugin=mesh_plugin, plot_tangent=True, scale=scale_cluster)
save_path = "render/images/unconditional/{}/{}.png".format("umap_clusters", meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show(fig_cluster)
if save_objs:
    label = "umap_clusters"
    meshes = ghd_reconstruct.ghd_forward_as_Meshes(ghd_cluster, trimmed_faces=mesh_plugin.trimmed_faces, mean=mean, std=std)
    save_dir = "render/meshes/unconditional/{}/{}".format(label, meta)
    os.makedirs(os.path.dirname(save_dir), exist_ok=True)
    mesh_render.save_objs(meshes, label, save_dir=save_dir)

In [ ]:
# near-neighbour generation
label='near_neighbour'
shape_id = 2
B = 12
ghd_real = next(iter(dataloader)).to(device)
if withscale:
    ghd_real, scale_real = ghd_real[:, :-1], ghd_real[:, -1:]
else:
    scale_real = None
mu, logvar = generator.encode(ghd_real, scale_real)
mu = mu[shape_id].unsqueeze(0).repeat(B, 1)
logvar = logvar[shape_id].unsqueeze(0).repeat(B, 1)
latent_std = torch.exp(logvar/2)
# z = mu + torch.randn_like(latent_std) * latent_std
if latent_dim > 64:
    z = mu + torch.randn_like(latent_std) * 0.5
else:
    z = mu + torch.randn_like(latent_std) * 0.5

if withscale:
    ghd_fake, scale_fake = generator.decode(z, False)
else:
    ghd_fake = generator.decode(z, True)
    scale_fake = None
data_fake = ghd_reconstruct.forward(ghd_fake, mean, std, use_norm)
fig = get_fig(ghd_reconstruct, data_fake, 5, Title="Near-Neighbour Generation")
save_path = "render/images/unconditional/{}/{}.png".format(label, meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show(fig)

# create gifs
meshes = ghd_reconstruct.ghd_forward_as_Meshes(ghd_fake * std.to(device) + mean.to(device), denormalize_shape=False, trimmed_faces=mesh_plugin.trimmed_faces)
if get_gif:
    mesh_render.render_rotational_gif(meshes, dual_texture=False, n_row=5, fps=36, out_dir=None, label=label)
    display(Image(filename=f"./render/gifs/{label}.gif"))
try:
    display(Image(filename=f"./render/gifs/{label}.gif"))
except:
    pass
if save_objs:
    save_dir = "render/meshes/unconditional/{}/{}".format(label, meta)
    os.makedirs(os.path.dirname(save_dir), exist_ok=True)
    mesh_render.save_objs(meshes, label, save_dir=save_dir)

In [ ]:
# interpolation
label='interpolation'
B = 10
shape_id_l = 7
shape_id_r = 2
ghd_real = next(iter(dataloader)).to(device)
if withscale:
    ghd_real, scale_real = ghd_real[:, :-1], ghd_real[:, -1:]
else:
    scale_real = None
mu, logvar = generator.encode(ghd_real, scale_real)
z_l = mu[shape_id_l].unsqueeze(0)
z_r = mu[shape_id_r].unsqueeze(0)
# z_l = -1 * torch.ones(1, latent_dim).to(device)
# z_r = 0 * torch.ones(1, latent_dim).to(device)
z = []
for i in range(B):
    alpha = i / (B - 1)
    z.append((1 - alpha) * z_l + alpha * z_r)
z = torch.cat(z, 0)
if withscale:
    ghd_fake, scale_fake = generator.decode(z, False)
    scale_fake = dataset.denorm_scale(scale_fake)
else:
    ghd_fake = generator.decode(z, True)
    scale_fake = None
data_fake = ghd_reconstruct.forward(ghd_fake, mean, std, use_norm)
fig = get_fig(ghd_reconstruct, data_fake, 5, Title="Interpolation", scale=scale_fake)
save_path = "render/images/unconditional/{}/{}.png".format(label, meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show(fig)

meshes = ghd_reconstruct.ghd_forward_as_Meshes(ghd_fake * std.to(device) + mean.to(device), denormalize_shape=False, trimmed_faces=mesh_plugin.trimmed_faces)
if get_gif:
    mesh_render.render_rotational_gif(meshes, dual_texture=False, n_row=5, fps=36, out_dir=None, label=label)
try:
    display(Image(filename=f"./render/gifs/{label}.gif"))
except:
    pass
if save_objs:
    save_dir = "render/meshes/unconditional/{}/{}".format(label, meta)
    os.makedirs(os.path.dirname(save_dir), exist_ok=True)
    mesh_render.save_objs(meshes, label, save_dir=save_dir)

In [ ]:
from pytorch3d.structures import Meshes
ghd_real = next(iter(dataloader)).to(device)
if withscale:
    ghd_real, scale_real = ghd_real[:, :-1], ghd_real[:, -1:]
else:
    scale_real = None
    
B = 10 * ghd_real.shape[0]
z = torch.randn(B, latent_dim).to(device)
if withscale:
    ghd_fake, scale_fake = generator.decode(z, False)
else:
    ghd_fake = generator.decode(z, True)
    scale_fake = None
data_real = ghd_reconstruct.forward(ghd_real, mean, std, use_norm)
data_fake = ghd_reconstruct.forward(ghd_fake, mean, std, use_norm)
data_real = data_real.to_data_list()
data_fake = data_fake.to_data_list()
reg_real = []
reg_fake = []
canonical_mesh = ghd_reconstruct.canonical_Meshes.to(device)
for data_real_ in data_real:
    reg_real.append(rigidloss(data_real_.pos))
for data_fake_ in data_fake:
    reg_fake.append(rigidloss(data_fake_.pos))

reg_real = torch.stack(reg_real).detach().cpu().numpy()
reg_fake = torch.stack(reg_fake).detach().cpu().numpy()
print(reg_real.shape, reg_fake.shape)
plt.figure(figsize=(10, 6))
data = [reg_real, reg_fake]
# for i, d in enumerate(data):
#     x = np.random.normal(loc=i + 1, scale=0.04, size=len(d))  # Add jitter to x-coordinates
#     plt.scatter(x, d, alpha=0.7, color='darkblue', edgecolor='white', s=20)
plt.violinplot(data, showmeans=True, showextrema=True)
plt.xticks([1, 2], ['Real', 'Fake'])
plt.ylabel('Rigid Loss')
plt.title('Distribution of Rigid Loss for Real and Fake Data')
save_path = "render/images/unconditional/{}/{}.png".format("rigid_distribution", meta)
os.makedirs(os.path.dirname(save_path), exist_ok=True)
plt.savefig(save_path)
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Parameters for the non-standard Gaussian distribution
mu = 0  # Mean
sigma = 2  # Standard deviation (makes the distribution wider and lower in height)
amplitude = 0.1  # Reduces the height of the distribution

# Generate data for the non-standard Gaussian distribution
x = np.linspace(-6, 6, 1000)
y = amplitude * (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

# Create a much lighter red-like gradient colormap for the fill
cmap = LinearSegmentedColormap.from_list('light_red_gradient', ['lavenderblush', 'lightcoral'])

# Create the plot
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the Gaussian curve with a black line
ax.plot(x, y, color='black', linewidth=2)

# Fill under the curve with the gradient color along the y-axis
for i in range(len(x) - 1):
    ax.fill_between(x[i:i+2], y[i:i+2], color=cmap(y[i] / y.max()), alpha=1)

# Remove axes
ax.axis('off')

# Show the plot
plt.savefig("render/images/unconditional/{}.png".format("gaussian2"))
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Parameters for the non-standard Gaussian distribution
mu = 0  # Mean
sigma = 2  # Standard deviation (makes the distribution wider and lower in height)
amplitude = 0.1  # Reduces the height of the distribution

# Generate data for the non-standard Gaussian distribution
x = np.linspace(-8, 8, 1000)
y = amplitude * (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

# Create a much lighter red-like gradient colormap for the fill
cmap = LinearSegmentedColormap.from_list('blue_gradient', ['aliceblue', 'skyblue'])

# Create the plot
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the Gaussian curve with a black line
ax.plot(x, y, color='black', linewidth=2)

# Fill under the curve with the gradient color along the y-axis
for i in range(len(x) - 1):
    ax.fill_between(x[i:i+2], y[i:i+2], color=cmap(y[i] / y.max()), alpha=1)

# Remove axes
ax.axis('off')

# Show the plot
plt.savefig("render/images/unconditional/{}.png".format("gaussian1"))
plt.show()

In [ ]:
import os
import shutil

original_root = '/media/yaplab/"HDD Storage"/wenhao/ghb_aneurysms/checkpoints/fit_mca_stable'
new_root = '/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/ghd_fitting'
folders = [dir for dir in os.listdir(original_root) if os.path.isdir(os.path.join(original_root, dir))]
file_list = ["ghb_fitting_checkpoint_5.pkl"]
for folder in folders:
    original_subfolder = os.path.join(original_root, folder, "vanilla")
    new_subfolder = os.path.join(new_root, folder, "vanilla")
    if not os.path.exists(original_subfolder):
        continue
    for file_ in file_list:
        if file_ in os.listdir(original_subfolder):
            os.makedirs(new_subfolder, exist_ok=True)
            shutil.copy2(os.path.join(original_subfolder, file_), os.path.join(new_subfolder, file_))
            print("{} has been copied".format(os.path.join(new_root, folder, file_)))


In [3]:
import os
import shutil

original_root = '/media/yaplab/"HDD Storage"/wenhao/ghb_aneurysms/checkpoints/align_mca_stable'
new_root = '/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment'
folders = [dir for dir in os.listdir(original_root) if os.path.isdir(os.path.join(original_root, dir))]
file_list = ["aligned_centreline.pkl", "alignment_checkpoint.pkl", "part_aligned_updated.obj", "diff_centreline_checkpoint.pkl",
             "opa_checkpoint.pkl", "part_aligned.obj", "part_aligned_updated.obj"]
for folder in folders:
    original_subfolder = os.path.join(original_root, folder)
    new_subfolder = os.path.join(new_root, folder)
    if not os.path.exists(original_subfolder):
        continue
    for file_ in file_list:
        if file_ in os.listdir(original_subfolder):
            os.makedirs(new_subfolder, exist_ok=True)
            shutil.copy2(os.path.join(original_subfolder, file_), os.path.join(new_subfolder, file_))
            print("{} has been copied".format(os.path.join(new_root, folder, file_)))


/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/aligned_centreline.pkl has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/alignment_checkpoint.pkl has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/part_aligned_updated.obj has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/diff_centreline_checkpoint.pkl has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/opa_checkpoint.pkl has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/part_aligned.obj has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/UPF_P0284.00_ID1/part_aligned_updated.obj has been copied
/media/yaplab/"HDD Storage"/wenhao/AneuG/checkpoints/alignment/p163_EwkdAxESDAkcBwcVEQkBBxQC/aligned_centreline.pkl has been copied
/media/yaplab/"HDD Storage"/wen